In [ ]:
# ============== AGENT 1: LLM-POWERED USER PROFILER (V3 - Corrected) =============="

import os
import json
import socket
from typing import List

# As per the deprecation warning, we should import from Pydantic V2.
# This future-proofs our code.
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_ollama.chat_models import ChatOllama

# --- Pydantic model for structured LLM output ---
class Questionnaire(BaseModel):
    questions: List[str] = Field(description="A list of 4-5 questions for the user.")

class UserProfilerAgent_V3:
    """
    Agent 1 (V3): Guarantees the user's name is collected first before
    using an LLM to dynamically generate the rest of the questionnaire.
    """

    def __init__(self, profiles_dir: str = "user_profiles"):
        self.profiles_dir = profiles_dir
        if not os.path.exists(self.profiles_dir):
            os.makedirs(self.profiles_dir)

        try:
            host_node = socket.gethostname()
            # NOTE: Replace 'jgarc111' with the ASURITE ID of the user running the Ollama server
            self.llm = ChatOllama(model="qwen3:14b", base_url=f"http://jgarc111@{host_node}:11434/")
            self.structured_llm = self.llm.with_structured_output(Questionnaire)
            print("✅ Successfully connected to Ollama LLM.")
        except Exception as e:
            print(f"❌ Error connecting to Ollama: {e}")
            self.llm = None

    def _generate_questions_with_llm(self) -> List[str]:
        """
        Uses an LLM to dynamically generate a user questionnaire.
        **Correction:** The prompt now explicitly tells the LLM to NOT ask for a name.
        """
        print("\n🤖 Generating a personalized questionnaire for you...")
        
        prompt = PromptTemplate(
            template="""
            You are a helpful assistant for an AI Data Science Tutor. Your goal is to create a short questionnaire (4-5 questions) to understand a user's knowledge level.
            
            The questions should gently probe their experience with:
            1. The Python programming language.
            2. Common CPU-based data science libraries (like NumPy, Pandas).
            3. Their awareness of GPU computing and hardware acceleration.
            4. Their familiarity with any NVIDIA-specific GPU libra_ries (like CuPy or RAPIDS).

            IMPORTANT: Do NOT ask for the user's name, as it has already been collected.
            Return the questions as a JSON list. Be conversational and friendly.
            """,
            input_variables=[],
        )
        
        query_generation_chain = prompt | self.structured_llm
        
        try:
            response_model = query_generation_chain.invoke({})
            return response_model.questions
        except Exception as e:
            print(f"-> LLM failed to generate questions, falling back to default. Error: {e}")
            return [
                "On a scale of 1-5, how comfortable are you with Python?",
                "Which Python data science libraries (like Pandas or NumPy) have you used before?",
                "Have you ever heard of using GPUs to speed up data analysis?",
                "What's the first tool you'd reach for to do a large matrix multiplication in Python?"
            ]

    def _ask_dynamic_questions(self, questions: List[str]) -> dict:
        """
        Asks the dynamically generated questions and records the answers.
        """
        answers = {}
        for i, question in enumerate(questions, start=1):
            answer = input(f"{i+1}. {question} ") # Start numbering from 2
            answers[question] = answer
        return answers

    def _generate_report_with_llm(self, user_name: str, answers: dict) -> str:
        """
        Sends the user's answers to the LLM to generate a profile report.
        """
        print("\n🤖 Analyzing your responses and creating a profile...")
        
        answers_str = "\n".join([f"- {q}: {a}" for q, a in answers.items()])

        prompt = PromptTemplate(
            template="""
            You are an expert AI analyst. A user named {user_name} has answered a questionnaire about their data science skills.
            Your task is to analyze their answers and generate a "TUTORING STRATEGY" report for our AI Tutor.

            **User's Answers:**
            {answers}

            **Your Task:**
            1.  Determine the user's knowledge level: 'Beginner', 'Intermediate', or 'Advanced'.
            2.  Write a concise report following the correct strategy format below. This report will be given to another AI, so the instructions must be clear.

            ---
            **STRATEGY FORMATS (Choose ONE):**

            **If 'Beginner':**
            Start with `Knowledge Level: Beginner` on one line. On the next line, start with the exact phrase `TUTORING STRATEGY: The user is a beginner.` Then, explain that the tutor should use high-level concepts, explain the 'why' of GPU acceleration, and introduce NVIDIA libraries (like CuPy) as a simple, powerful alternative to what they already know.

            **If 'Intermediate':**
            Start with `Knowledge Level: Intermediate` on one line. On the next line, start with the exact phrase `TUTORING STRATEGY: The user is at an intermediate level.` Then, explain that the tutor should provide direct code comparisons (e.g., NumPy vs. CuPy), focus on performance benefits, and show clear benchmarking examples.

            **If 'Advanced':**
            Start with `Knowledge Level: Advanced` on one line. On the next line, start with the exact phrase `TUTORING STRATEGY: The user is advanced.` Then, explain that the tutor can provide nuanced advice, discuss the broader NVIDIA RAPIDS ecosystem, and cover specific benchmarking methodologies on the Sol supercomputer.
            ---

            Now, generate the complete report, including the "Knowledge Level" and the "TUTORING STRATEGY".
            """,
            input_variables=["user_name", "answers"],
        )

        report_generation_chain = prompt | self.llm
        response_message = report_generation_chain.invoke({"user_name": user_name, "answers": answers_str})
        report_content = response_message.content
        
        return report_content

    def _save_report(self, report: str, user_name: str):
        """
        Saves the generated report to a text file.
        """
        filename = "_".join(user_name.lower().split()) + ".txt"
        filepath = os.path.join(self.profiles_dir, filename)
        
        with open(filepath, "w") as f:
            f.write(report)
            
        print(f"\n✅ User profile report saved successfully to: {filepath}")

    def run(self):
        """
        The main method to run the agent's full workflow.
        **Correction:** Asks for name first to ensure correct file naming.
        """
        if not self.llm:
            return

        print("🤖 Hello! I'm your AI Accelerated Data Science Tutor.")
        user_name = input("1. To get started, what is your first and last name? ")

        questions = self._generate_questions_with_llm()
        answers = self._ask_dynamic_questions(questions)
        report_content = self._generate_report_with_llm(user_name, answers)
        
        # Add the final header to the report
        full_report = f"--- User Profile for {user_name} ---\n{report_content}\n--- End of Profile ---"
        
        print("\n--- Generated Profile Report ---")
        print(full_report)
        
        self._save_report(full_report, user_name)

# --- Example Usage ---
if __name__ == '__main__':
    profiler_agent = UserProfilerAgent_V3()
    profiler_agent.run()

✅ Successfully connected to Ollama LLM.
🤖 Hello! I'm your AI Accelerated Data Science Tutor.


In [ ]:
# ============== AGENT 1: GRADIO INTERFACE FOR USER PROFILER =============="
# This script combines the LLM-powered agent with a user-friendly Gradio UI.

import os
import json
import socket
import gradio as gr
from typing import List, Dict, Any

# Pydantic and LangChain imports
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_ollama.chat_models import ChatOllama

# --- Pydantic Models for Structured LLM Output ---
class Questionnaire(BaseModel):
    """A Pydantic model to structure the questionnaire from the LLM."""
    questions: List[str] = Field(description="A list of 4-5 questions for the user.")

class ProfileAnalysis(BaseModel):
    """A Pydantic model to structure the profile analysis from the LLM."""
    knowledge_level: str = Field(description="The user's determined knowledge level: 'Beginner', 'Intermediate', or 'Advanced'.")
    tutoring_strategy: str = Field(description="The detailed tutoring strategy based on the user's knowledge level.")

# --- The Backend Agent Logic ---# ============== AGENT 1: GRADIO INTERFACE FOR USER PROFILER =============="
# This script combines the LLM-powered agent with a user-friendly Gradio UI.

import os
import json
import socket
import gradio as gr
from typing import List, Dict, Any

# Pydantic and LangChain imports
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_ollama.chat_models import ChatOllama

# --- Pydantic Models for Structured LLM Output ---
class Questionnaire(BaseModel):
    """A Pydantic model to structure the questionnaire from the LLM."""
    questions: List[str] = Field(description="A list of 4-5 questions for the user.")

class ProfileAnalysis(BaseModel):
    """A Pydantic model to structure the profile analysis from the LLM."""
    knowledge_level: str = Field(description="The user's determined knowledge level: 'Beginner', 'Intermediate', or 'Advanced'.")
    tutoring_strategy: str = Field(description="The detailed tutoring strategy based on the user's knowledge level.")

# --- The Backend Agent Logic ---
class UserProfilerAgent_Gradio:
    """
    Agent 1 (Gradio Version): Handles the logic for profiling users via an LLM.
    This class is designed to be called by the Gradio interface functions.
    """
    def __init__(self, profiles_dir: str = "user_profiles"):
        self.profiles_dir = profiles_dir
        if not os.path.exists(self.profiles_dir):
            os.makedirs(self.profiles_dir)

        try:
            host_node = socket.gethostname()
            # IMPORTANT: Replace 'jgarc111' with the ASURITE ID of the user running the Ollama server on Sol.
            self.llm = ChatOllama(model="qwen3:14b", temperature=0, base_url=f"http://jgarc111@{host_node}:11434/")
            print("✅ Successfully connected to Ollama LLM.")
        except Exception as e:
            print(f"❌ Error connecting to Ollama: {e}")
            self.llm = None

    def generate_questions(self) -> List[str]:
        """Uses an LLM to dynamically generate the questionnaire."""
        if not self.llm: return self._fallback_questions()

        prompt = PromptTemplate.from_template(
            """
            You are an assistant for an AI Data Science Tutor. Create a short questionnaire (4-5 questions)
            to understand a user's knowledge of Python, data science libraries (NumPy, Pandas), and their
            awareness of GPU computing (NVIDIA, CuPy, RAPIDS).
            IMPORTANT: Do NOT ask for the user's name. Return a JSON list of questions.
            """
        )
        structured_llm = self.llm.with_structured_output(Questionnaire)
        chain = prompt | structured_llm
        try:
            return chain.invoke({}).questions
        except Exception:
            return self._fallback_questions()

    def _fallback_questions(self) -> List[str]:
        """Fallback questions if the LLM fails."""
        return [
            "On a scale of 1-5, how comfortable are you with Python?",
            "Which Python data science libraries (like Pandas or NumPy) have you used before?",
            "Have you ever heard of using GPUs to speed up data analysis?",
            "What's the first tool you'd reach for to do a large matrix multiplication in Python?"
        ]

    def generate_report(self, user_name: str, answers: Dict[str, str]) -> str:
        """Uses an LLM to analyze answers and generate a profile report."""
        if not self.llm: return "Error: LLM not connected."

        answers_str = "\n".join([f"- {q}: {a}" for q, a in answers.items()])
        prompt = PromptTemplate.from_template(
            """
            You are an expert AI analyst. A user named {user_name} has answered a questionnaire.
            Analyze their answers and generate a profile.

            **User's Answers:**
            {answers}

            **Your Task:**
            1.  Determine the user's knowledge level: 'Beginner', 'Intermediate', or 'Advanced'.
            2.  Write a concise tutoring strategy for our AI Tutor based on this level.
            
            **Example for Intermediate:** Start with a strategy focusing on direct code comparisons (NumPy vs. CuPy) and performance benefits.
            Provide your output as a JSON object with 'knowledge_level' and 'tutoring_strategy' fields.
            """
        )
        structured_llm = self.llm.with_structured_output(ProfileAnalysis)
        chain = prompt | structured_llm
        try:
            analysis = chain.invoke({"user_name": user_name, "answers": answers_str})
            report_content = f"Knowledge Level: {analysis.knowledge_level}\n\nTUTORING STRATEGY: {analysis.tutoring_strategy}"
            full_report = f"--- User Profile for {user_name} ---\n{report_content}\n--- End of Profile ---"
            return full_report
        except Exception as e:
            return f"Error generating report: {e}"

    def save_report(self, report: str, user_name: str) -> str:
        """Saves the report to a file, overwriting if it exists."""
        filename = "_".join(user_name.lower().split()) + ".txt"
        filepath = os.path.join(self.profiles_dir, filename)
        with open(filepath, "w") as f:
            f.write(report)
        return f"✅ Profile for {user_name} saved successfully to: {filepath}"

# --- Gradio UI Application ---

# Instantiate the agent
agent = UserProfilerAgent_Gradio()

def start_workflow(mode: str):
    """Hides main buttons and shows the name input field."""
    initial_state = {"mode": mode, "name": "", "questions": [], "answers": {}, "current_q_index": -1}
    return (
        gr.update(visible=False), # Hide New User button
        gr.update(visible=False), # Hide Update Profile button
        gr.update(visible=True),  # Show name row
        initial_state
    )

def process_name(name: str, state: Dict[str, Any]):
    """Processes the user's name and gets the questionnaire."""
    if not name.strip():
        return gr.update(), gr.update(), gr.update(), gr.update(), state # No change if name is empty

    state["name"] = name
    questions = agent.generate_questions()
    state["questions"] = questions
    state["current_q_index"] = 0
    
    return (
        gr.update(visible=False), # Hide name row
        gr.update(visible=True),  # Show question row
        gr.update(label=questions[0]), # Update question label
        gr.update(value=""), # Clear answer textbox
        state
    )

def process_answer(answer: str, state: Dict[str, Any]):
    """Processes an answer, shows the next question, or finishes the quiz."""
    # Save the last answer
    current_question = state["questions"][state["current_q_index"]]
    state["answers"][current_question] = answer
    
    # Move to the next question
    state["current_q_index"] += 1
    
    if state["current_q_index"] < len(state["questions"]):
        # Still more questions
        next_question = state["questions"][state["current_q_index"]]
        return (
            gr.update(label=next_question),
            gr.update(value=""),
            gr.update(), # No change to final report
            gr.update(), # No change to buttons
            gr.update(),
            state
        )
    else:
        # Finished questionnaire
        report = agent.generate_report(state["name"], state["answers"])
        save_status = agent.save_report(report, state["name"])
        final_display = f"{report}\n\n{save_status}"
        
        return (
            gr.update(),
            gr.update(),
            gr.update(value=final_display, visible=True), # Show final report
            gr.update(visible=False), # Hide question row
            gr.update(visible=True),  # Show main buttons again
            state
        )

def reset_ui():
    """Resets the UI to its initial state."""
    return (
        gr.update(visible=True),  # Show New User
        gr.update(visible=True),  # Show Update Profile
        gr.update(visible=False), # Hide name row
        gr.update(visible=False), # Hide question row
        gr.update(visible=False, value="") # Hide final report
    )

# --- Build the Gradio Interface using Blocks ---
with gr.Blocks(theme=gr.themes.Soft(), css=".gradio-container {background-color: #f5f5f5;}") as demo:
    gr.Markdown("# 🤖 Agent 1: User Profiler")
    gr.Markdown("Create a new user profile or update an existing one. This helps the AI Tutor tailor its explanations to your skill level.")
    
    # State object to hold session data
    session_state = gr.State(value={})
    
    # --- UI Components ---
    with gr.Row() as main_buttons_row:
        new_user_btn = gr.Button("👋 New User", variant="primary")
        update_user_btn = gr.Button("🔄 Update Profile / Retake", variant="secondary")

    with gr.Row(visible=False) as name_row:
        name_input = gr.Textbox(label="What is your first and last name?", placeholder="e.g., Jane Doe")
        name_submit_btn = gr.Button("Submit Name")

    with gr.Row(visible=False) as question_row:
        answer_input = gr.Textbox(label="Your Answer", placeholder="Type your answer here...")
        answer_submit_btn = gr.Button("Submit Answer")
        
    final_report_display = gr.Markdown(visible=False)

    # --- Event Handlers ---
    new_user_btn.click(
        fn=lambda: start_workflow("new"),
        outputs=[new_user_btn, update_user_btn, name_row, session_state]
    )
    update_user_btn.click(
        fn=lambda: start_workflow("update"),
        outputs=[new_user_btn, update_user_btn, name_row, session_state]
    )
    
    # Logic for when a name is submitted
    name_submit_btn.click(
        fn=process_name,
        inputs=[name_input, session_state],
        outputs=[name_row, question_row, answer_input, answer_input, session_state]
    )
    
    # Logic for when an answer is submitted
    answer_submit_btn.click(
        fn=process_answer,
        inputs=[answer_input, session_state],
        outputs=[answer_input, answer_input, final_report_display, question_row, main_buttons_row, session_state]
    ).then(
        fn=reset_ui,
        outputs=[new_user_btn, update_user_btn, name_row, question_row, final_report_display]
    )

# --- Launch the application ---
# Set share=True to get a public link when running on Sol's Jupyter notebooks.
if __name__ == "__main__":
    demo.launch(share=True, debug=True)

class UserProfilerAgent_Gradio:
    """
    Agent 1 (Gradio Version): Handles the logic for profiling users via an LLM.
    This class is designed to be called by the Gradio interface functions.
    """
    def __init__(self, profiles_dir: str = "user_profiles"):
        self.profiles_dir = profiles_dir
        if not os.path.exists(self.profiles_dir):
            os.makedirs(self.profiles_dir)

        try:
            host_node = socket.gethostname()
            # IMPORTANT: Replace 'jgarc111' with the ASURITE ID of the user running the Ollama server on Sol.
            self.llm = ChatOllama(model="qwen3:14b", temperature=0, base_url=f"http://jgarc111@{host_node}:11434/")
            print("✅ Successfully connected to Ollama LLM.")
        except Exception as e:
            print(f"❌ Error connecting to Ollama: {e}")
            self.llm = None

    def generate_questions(self) -> List[str]:
        """Uses an LLM to dynamically generate the questionnaire."""
        if not self.llm: return self._fallback_questions()

        prompt = PromptTemplate.from_template(
            """
            You are an assistant for an AI Data Science Tutor. Create a short questionnaire (4-5 questions)
            to understand a user's knowledge of Python, data science libraries (NumPy, Pandas), and their
            awareness of GPU computing (NVIDIA, CuPy, RAPIDS).
            IMPORTANT: Do NOT ask for the user's name. Return a JSON list of questions.
            """
        )
        structured_llm = self.llm.with_structured_output(Questionnaire)
        chain = prompt | structured_llm
        try:
            return chain.invoke({}).questions
        except Exception:
            return self._fallback_questions()

    def _fallback_questions(self) -> List[str]:
        """Fallback questions if the LLM fails."""
        return [
            "On a scale of 1-5, how comfortable are you with Python?",
            "Which Python data science libraries (like Pandas or NumPy) have you used before?",
            "Have you ever heard of using GPUs to speed up data analysis?",
            "What's the first tool you'd reach for to do a large matrix multiplication in Python?"
        ]

    def generate_report(self, user_name: str, answers: Dict[str, str]) -> str:
        """Uses an LLM to analyze answers and generate a profile report."""
        if not self.llm: return "Error: LLM not connected."

        answers_str = "\n".join([f"- {q}: {a}" for q, a in answers.items()])
        prompt = PromptTemplate.from_template(
            """
            You are an expert AI analyst. A user named {user_name} has answered a questionnaire.
            Analyze their answers and generate a profile.

            **User's Answers:**
            {answers}

            **Your Task:**
            1.  Determine the user's knowledge level: 'Beginner', 'Intermediate', or 'Advanced'.
            2.  Write a concise tutoring strategy for our AI Tutor based on this level.
            
            **Example for Intermediate:** Start with a strategy focusing on direct code comparisons (NumPy vs. CuPy) and performance benefits.
            Provide your output as a JSON object with 'knowledge_level' and 'tutoring_strategy' fields.
            """
        )
        structured_llm = self.llm.with_structured_output(ProfileAnalysis)
        chain = prompt | structured_llm
        try:
            analysis = chain.invoke({"user_name": user_name, "answers": answers_str})
            report_content = f"Knowledge Level: {analysis.knowledge_level}\n\nTUTORING STRATEGY: {analysis.tutoring_strategy}"
            full_report = f"--- User Profile for {user_name} ---\n{report_content}\n--- End of Profile ---"
            return full_report
        except Exception as e:
            return f"Error generating report: {e}"

    def save_report(self, report: str, user_name: str) -> str:
        """Saves the report to a file, overwriting if it exists."""
        filename = "_".join(user_name.lower().split()) + ".txt"
        filepath = os.path.join(self.profiles_dir, filename)
        with open(filepath, "w") as f:
            f.write(report)
        return f"✅ Profile for {user_name} saved successfully to: {filepath}"

# --- Gradio UI Application ---

# Instantiate the agent
agent = UserProfilerAgent_Gradio()

def start_workflow(mode: str):
    """Hides main buttons and shows the name input field."""
    initial_state = {"mode": mode, "name": "", "questions": [], "answers": {}, "current_q_index": -1}
    return (
        gr.update(visible=False), # Hide New User button
        gr.update(visible=False), # Hide Update Profile button
        gr.update(visible=True),  # Show name row
        initial_state
    )

def process_name(name: str, state: Dict[str, Any]):
    """Processes the user's name and gets the questionnaire."""
    if not name.strip():
        return gr.update(), gr.update(), gr.update(), gr.update(), state # No change if name is empty

    state["name"] = name
    questions = agent.generate_questions()
    state["questions"] = questions
    state["current_q_index"] = 0
    
    return (
        gr.update(visible=False), # Hide name row
        gr.update(visible=True),  # Show question row
        gr.update(label=questions[0]), # Update question label
        gr.update(value=""), # Clear answer textbox
        state
    )

def process_answer(answer: str, state: Dict[str, Any]):
    """Processes an answer, shows the next question, or finishes the quiz."""
    # Save the last answer
    current_question = state["questions"][state["current_q_index"]]
    state["answers"][current_question] = answer
    
    # Move to the next question
    state["current_q_index"] += 1
    
    if state["current_q_index"] < len(state["questions"]):
        # Still more questions
        next_question = state["questions"][state["current_q_index"]]
        return (
            gr.update(label=next_question),
            gr.update(value=""),
            gr.update(), # No change to final report
            gr.update(), # No change to buttons
            gr.update(),
            state
        )
    else:
        # Finished questionnaire
        report = agent.generate_report(state["name"], state["answers"])
        save_status = agent.save_report(report, state["name"])
        final_display = f"{report}\n\n{save_status}"
        
        return (
            gr.update(),
            gr.update(),
            gr.update(value=final_display, visible=True), # Show final report
            gr.update(visible=False), # Hide question row
            gr.update(visible=True),  # Show main buttons again
            state
        )

def reset_ui():
    """Resets the UI to its initial state."""
    return (
        gr.update(visible=True),  # Show New User
        gr.update(visible=True),  # Show Update Profile
        gr.update(visible=False), # Hide name row
        gr.update(visible=False), # Hide question row
        gr.update(visible=False, value="") # Hide final report
    )

# --- Build the Gradio Interface using Blocks ---
with gr.Blocks(theme=gr.themes.Soft(), css=".gradio-container {background-color: #f5f5f5;}") as demo:
    gr.Markdown("# 🤖 Agent 1: User Profiler")
    gr.Markdown("Create a new user profile or update an existing one. This helps the AI Tutor tailor its explanations to your skill level.")
    
    # State object to hold session data
    session_state = gr.State(value={})
    
    # --- UI Components ---
    with gr.Row() as main_buttons_row:
        new_user_btn = gr.Button("👋 New User", variant="primary")
        update_user_btn = gr.Button("🔄 Update Profile / Retake", variant="secondary")

    with gr.Row(visible=False) as name_row:
        name_input = gr.Textbox(label="What is your first and last name?", placeholder="e.g., Jane Doe")
        name_submit_btn = gr.Button("Submit Name")

    with gr.Row(visible=False) as question_row:
        answer_input = gr.Textbox(label="Your Answer", placeholder="Type your answer here...")
        answer_submit_btn = gr.Button("Submit Answer")
        
    final_report_display = gr.Markdown(visible=False)

    # --- Event Handlers ---
    new_user_btn.click(
        fn=lambda: start_workflow("new"),
        outputs=[new_user_btn, update_user_btn, name_row, session_state]
    )
    update_user_btn.click(
        fn=lambda: start_workflow("update"),
        outputs=[new_user_btn, update_user_btn, name_row, session_state]
    )
    
    # Logic for when a name is submitted
    name_submit_btn.click(
        fn=process_name,
        inputs=[name_input, session_state],
        outputs=[name_row, question_row, answer_input, answer_input, session_state]
    )
    
    # Logic for when an answer is submitted
    answer_submit_btn.click(
        fn=process_answer,
        inputs=[answer_input, session_state],
        outputs=[answer_input, answer_input, final_report_display, question_row, main_buttons_row, session_state]
    ).then(
        fn=reset_ui,
        outputs=[new_user_btn, update_user_btn, name_row, question_row, final_report_display]
    )

# --- Launch the application ---
# Set share=True to get a public link when running on Sol's Jupyter notebooks.
if __name__ == "__main__":
    demo.launch(share=True, debug=True)


✅ Successfully connected to Ollama LLM.
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://c9693d375e12c96772.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
